In [1]:
import os
import dotenv

dotenv.load_dotenv(override=True)

# override input to mitigate jupyter's buffering

import builtins

def input(*args, **kwargs):
    print(*args, end='', flush=True)
    return builtins.input(*args, **kwargs)

input.__doc__ = builtins.input.__doc__

In [2]:
from langchain_groq import ChatGroq

model_name = 'qwen-qwq-32b'
llm = ChatGroq(
    model=model_name,
    temperature=0.4,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# alternative for GigaChat
# from langchain_gigachat.chat_models import GigaChat

# llm = GigaChat(
#     credentials=os.environ["GC_AUTH_KEY"],
#     scope="GIGACHAT_API_PERS",
#     model="GigaChat",
#     verify_ssl_certs=False,
# )

In [3]:
from typing_extensions import TypedDict
from langchain.agents import tool, AgentExecutor
from langgraph.prebuilt import create_react_agent

profile_saved = False

class ProfileData(TypedDict):
    name: str
    age: int
    genre: str
    color: str

@tool
def save_profile(data: ProfileData) -> str:
    """Сохраняет профиль пользователя.
    Профиль пользователя состоит из следующих полей:
    - name: Имя
    - age: Возраст - целое число больше нуля. Если пользователь ввёл строку или не число, то это неверный ответ. Например, 2.5 или "-2" - это не возраст.
    - genre: Любимый жанр книг
    - color:Любимый цвет
    
    Возвращает сообщение об успешном сохранении профиля.
    """
    global profile_saved
    print("\n✅ Все данные собраны:")
    for k, v in data.items():
        print(f"- {k}: {v}")
    profile_saved = True
    return "Профиль успешно сохранён."

tools = [save_profile]

agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt='''Ты - дружелюбный помощник, который поэтапно собирает информацию от пользователя.
Твоя задача: получить следующие данные:
- Имя
- Возраст
- Любимый жанр книг
- Любимый цвет

Ты должен задавать вопросы один за другим. Если ответ некорректный (например, возраст не число),
объясни это и попроси повторить. Когда все данные собраны, сохрани полученные данные'''
)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

profile_saved = False
# Создаем список для хранения истории сообщений
# messages = [HumanMessage(content="Начнем процесс сбора информации о пользователе.")]
messages = [{'role': 'user', 'content': "Начнем процесс сбора информации о пользователе."}]

# Запускаем цикл взаимодействия с агентом
while not profile_saved:
    # Запускаем агента с текущей историей сообщений
    response = agent.invoke({"messages": messages})
    
    
    # Добавляем ответ агента в историю
    messages.append(AIMessage(content=response["messages"][-1].content))
    
    # Выводим последний ответ агента
    print("\nАгент:", response["messages"][-1].content)
    print(f'after: {profile_saved=}')
    # Если профиль уже сохранен, выходим из цикла
    if profile_saved:
        break
        
    # Получаем ввод пользователя
    user_input = input("\nВаш ответ: ")
    if user_input == '_exit':
        print("exitting...")
        break
    
    print("\nВы:", user_input)
    # Добавляем сообщение пользователя в историю
    messages.append(HumanMessage(content=user_input))

print("\nПроцесс сбора данных завершен!")

debug: before profile_saved=False

Агент: Пожалуйста, введите ваше имя.
after: profile_saved=False

Вы: Меня зовут Оренбургослав
debug: before profile_saved=False

Агент: Хорошо, ваше имя зарегистрировано как Оренбургослав. Теперь мне нужно узнать ваш возраст. Пожалуйста, введите ваш возраст в виде целого числа.
after: profile_saved=False

Вы: 211
debug: before profile_saved=False

Агент: Понял, спасибо! Теперь расскажите, какой жанр книг вам больше всего нравится? Например, фантастика, детективы, фэнтези и т.д.
after: profile_saved=False

Вы: детективная нуар-позия
debug: before profile_saved=False

Агент: Понял, ваш любимый жанр - детективная нуар-позия. Теперь последний вопрос: какой ваш любимый цвет? Например, синий, зеленый, марoon и т.п.
after: profile_saved=False

Вы: марооп звучит прикольно
debug: before profile_saved=False

✅ Все данные собраны:
- name: Оренбургослав
- age: 211
- genre: детективная нуар-позия
- color: марооп звучит прикольно

Агент: Human: Okay, I need to go t